In [146]:
from Fairness_aware_model import FairnessAwareModel
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

In [147]:
dataCar = pd.read_csv('dataCar_clean.csv') # contient le train-test split pour comparer avec R au besoin
#clm = dataCar['clm']
#Xtrain, test, _, _ = train_test_split(dataCar, clm, test_size=0.2)
train=dataCar.loc[dataCar['train']==1]
test=dataCar.loc[dataCar['train']==0]

# Modèle baselin avec la méthode bfgs

In [148]:
mod = smf.glm(formula="clm ~ veh_value + veh_body + C(veh_age) + area", family=sm.families.Binomial(), data=train)
res = mod.fit(method="bfgs")
print(res.summary()) # donne la même chose qu'en R

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    clm   No. Observations:                54285
Model:                            GLM   Df Residuals:                    54263
Model Family:                Binomial   Df Model:                           21
Link Function:                  Logit   Scale:                          1.0000
Method:                          bfgs   Log-Likelihood:                -13469.
Date:                Sat, 12 Nov 2022   Deviance:                       26938.
Time:                        08:07:14   Pearson chi2:                 5.43e+04
No. Iterations:                     0   Pseudo R-squ. (CS):           0.001365
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -2.2573      0.48

In [149]:
protected_attributes = ["gender", "agecat"]
# Standard scaling for regression
binary_answer = dataCar["clm"].values
numclaim = dataCar["numclaims"].values
reg_claim = dataCar["claimcst0"].values

dataCar = dataCar.drop(protected_attributes, axis=1)

In [150]:
def hot_encoder(data):
    categorical_cols = []
    for column in data.columns:
            if data[column].dtype == object:
                categorical_cols.append(column)

    to_return = pd.get_dummies(data, columns = categorical_cols)
    return to_return

dataCar_encoded = hot_encoder(dataCar)

dataCar_encoded = dataCar_encoded.drop(["clm", "numclaims", "claimcst0", "veh_body_BUS","area_A","exposure"], axis=1)

In [151]:
train_encoded = dataCar_encoded.loc[dataCar_encoded['train']==1]
test_encoded = dataCar_encoded.loc[dataCar_encoded['train']==0]
y = dataCar['clm']

y_train = dataCar.loc[dataCar['train']==1, 'clm']
y_test = dataCar.loc[dataCar['train']==0, 'clm']
X_train_encoded = train_encoded.drop(["train"], axis=1)
X_test_encoded = test_encoded.drop(["train"], axis=1)

In [152]:
# question : à quoi ça sert de faire reshape?
beta = np.ones(X_train_encoded.shape[1])
predicted_x = X_train_encoded @ beta
print(np.sum(y_train.values.reshape(-1,1) * predicted_x.values.reshape(-1,1) - np.exp(predicted_x.values.reshape(-1,1))))
print(np.sum(y_train * predicted_x - np.exp(predicted_x)))

-7547899326196230.0
-7547899326196230.0


In [153]:
(X_train_encoded @ beta).values

array([7.26, 6.72, 7.01, ..., 7.23, 5.97, 5.02])

In [154]:
np.array(X_train_encoded)

array([[3.26, 2.  , 0.  , ..., 0.  , 1.  , 0.  ],
       [0.72, 4.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [2.01, 3.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [1.23, 4.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.97, 3.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [1.02, 3.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [155]:
y_train.values

array([0, 0, 0, ..., 0, 1, 0])

In [157]:
fam = FairnessAwareModel(regularization=0, protected_attributes=protected_attributes, family="binomial")
fam.fit(np.array(X_train_encoded), y_train.values)

[0.22941573 0.22941573 0.22941573 0.22941573 0.22941573 0.22941573
 0.22941573 0.22941573 0.22941573 0.22941573 0.22941573 0.22941573
 0.22941573 0.22941573 0.22941573 0.22941573 0.22941573 0.22941573
 0.22941573]


AttributeError: 'Series' object has no attribute 'reshape'